In [1]:
import numpy as np
import pandas as pd
import julian
import random
import sys
import os

sys.path.append(os.path.abspath('../../data_processing'))
from process_urls import get_file_urls, process_urls_in_parallel
from utils import plot_polar_data

In [2]:
temp_home = 'https://pds-geosciences.wustl.edu/lro/urn-nasa-pds-lro_diviner_derived1/data_derived_gdr_l3/2016/polar/jp2/'

# Get and print all .lbl file URLs from the given page
temp_urls = get_file_urls(temp_home, '.lbl', 'tbol', limit=4)
print(f"Found {len(temp_urls)} temp files")

# temp_urls = temp_urls[:5]

# selected_indices = random.sample(range(len(temp_urls)), 5)
# print(f"Selected indices: {selected_indices}")
# temp_urls = [temp_urls[i] for i in selected_indices]

print("Processing temp files...")
temp_df = process_urls_in_parallel(temp_urls, 'Diviner')
print("Files processed successfully")

Found 5 temp files
Processing temp files...
Extracting image data...Extracting image data...Extracting image data...


Extracting image data...
Image data extracted
sample projection offset: 1905.5, line projection offset: 1905.5, samples: 3812, lines: 3812, map resolution: 126.347
center_lat: 90.0, center_lon: 0.0, map_resolution: 126.347, min_lat: 75.0, max_lat: 90.0
Lats min: 89.29664208703777, max: 89.99981543774706
Lats min: 75.0, max: 90.0, Lons min: -179.98496568402047, max: 179.98496568402047
Coordinates generated
Number of Lats between -85 and -90 (1): 0
Extracting image data...
Image data extracted
sample projection offset: 1905.5, line projection offset: 1905.5, samples: 3812, lines: 3812, map resolution: 126.347
center_lat: 90.0, center_lon: 0.0, map_resolution: 126.347, min_lat: 75.0, max_lat: 90.0
Lats min: 89.29664208703777, max: 89.99981543774706
Lats min: 75.0, max: 90.0, Lons min: -179.98496568402047, max: 179.98496568402047
Coordinates generated
Number of Lats betwee

: 

In [3]:
print(f"Number of NaNs in Temperatures: {np.isnan(temp_df["Diviner"]).sum()} out of {np.prod(temp_df["Diviner"].shape)} ({(np.isnan(temp_df["Diviner"]).sum()/np.prod(temp_df["Diviner"].shape)*100):.2f}%)\n")

print(temp_df.describe())

plot_polar_data(temp_df, 'Diviner', frac=0.75, title_prefix='Diviner temperature')

Number of NaNs in Temperatures: 11638135 out of 72656720 (16.02%)

          Longitude      Latitude       Diviner
count  7.265672e+07  7.265672e+07  6.101858e+07
mean   1.799995e+02  8.188338e+01  1.714338e+02
std    1.034039e+02  3.022930e+00  8.406357e+01
min    1.503432e-02  7.500000e+01  3.754000e+01
25%    9.000752e+01  7.963361e+01  8.154000e+01
50%    1.800000e+02  8.153659e+01  1.949200e+02
75%    2.699925e+02  8.401658e+01  2.503200e+02
max    3.599850e+02  9.000000e+01  3.512600e+02
